In [158]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Load the dataset
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data = pd.read_csv('dataset.csv')
# Basic preprocessing
data = data.dropna()
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# Describing our data
We have data about some telecom company customers, we have data about:
- Our customer: customer id(will probably drop this column as it is irrelevant), type of contract, payment method, how much he pays monthly, how much did he/she paid overall.
- Our customer personal details such as gender, is he a senior or not, has dependents.
- We have data about whatever the customer is using specific telecom services such as Phone Service, StreamingTV, Streaming Movies and so on.
- and whatever he stopped using our services within our last month `Churn`

# Preprocessing our data
We will drop the customer id column and we will encode the rest of the data.

In [159]:
data = data.drop('customerID', axis=1)

# Encode Gender with One Hot encoder
one_hot_encoder = OneHotEncoder()
encoded = one_hot_encoder.fit_transform(data[['gender']])
gender_encoded = pd.DataFrame(encoded.toarray(), columns=one_hot_encoder.get_feature_names_out(['gender']))

# Encode MultipleLines, PaymentMethod, Contract and InternetService with Label Encoder
label_encoder = LabelEncoder()
data['Contract'] = label_encoder.fit_transform(data['Contract'])
data['PaymentMethod'] = label_encoder.fit_transform(data['PaymentMethod'])
data['MultipleLines'] = label_encoder.fit_transform(data['MultipleLines'])
data['InternetService'] = label_encoder.fit_transform(data['InternetService'])

# The rest of the columns, I will Binary Encode (Yes->1, No ->0)
for column in ['Partner', 'Dependents', 'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']:
    # At first I thought that simple replacing of Yes and No with 1 and 0 is enought, but there is a third value No Service Provided
    data[column] = label_encoder.fit_transform(data[column])

data = pd.concat([data.drop(columns=['gender']), gender_encoded], axis=1)

data.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male
0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0,1.0,0.0
1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.5,0,0.0,1.0
2,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1,0.0,1.0
3,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0,0.0,1.0
4,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1,1.0,0.0


In [160]:
data.describe(include='all')

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043,7043.000000,7043.000000,7043.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6531,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN,NaN,NaN
mean,0.162147,0.483033,0.299588,32.371149,0.903166,0.940508,0.872923,0.790004,0.906432,0.904444,0.797104,0.985376,0.992475,0.690473,0.592219,1.574329,64.761692,NaN,0.265370,0.495244,0.504756
std,0.368612,0.499748,0.458110,24.559481,0.295752,0.948554,0.737796,0.859848,0.880162,0.879949,0.861551,0.885002,0.885091,0.833755,0.491457,1.068104,30.090047,NaN,0.441561,0.500013,0.500013
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,NaN,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,35.500000,NaN,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,2.000000,70.350000,NaN,0.000000,0.000000,1.000000
75%,0.000000,1.000000,1.000000,55.000000,1.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,89.850000,NaN,1.000000,1.000000,1.000000


In [161]:
# We will replace empty strings with 0, since there are minor errors in this csv 
data = data.apply(pd.to_numeric, errors='coerce')
data.fillna(0, inplace=True)

# Splitting the data

In [162]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Churn', axis=1),
                                                         data['Churn'], 
                                                    test_size=0.4, 
                                                    random_state=42)


In [163]:
data.head() 

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male
0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0,1.0,0.0
1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0,0.0,1.0
2,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1,0.0,1.0
3,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0,0.0,1.0
4,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1,1.0,0.0


# Training and predicting using Naive Baise models
We will naive baise models to predict if a customer can leave.

In [164]:
# Create instances 
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
gaussian_bayes = GaussianNB()
multinomail_bayes = MultinomialNB()
bernoulli_bayes = BernoulliNB()

# Training
gaussian_bayes.fit(X_train, y_train)
multinomail_bayes.fit(X_train, y_train)
bernoulli_bayes.fit(X_train, y_train)

# Predict
predicted_churn_gaussian_bayes = gaussian_bayes.predict(X_test)
predicted_churn_multinomail_bayes = multinomail_bayes.predict(X_test)
predicted_churn_bernoulli_bayes = bernoulli_bayes.predict(X_test)


## Getting the accurancy for our data

In [165]:
from sklearn.metrics import accuracy_score

gaussian_bayes_accuracy = accuracy_score(y_test, predicted_churn_gaussian_bayes)
multinomail_bayes_accuracy = accuracy_score(y_test, predicted_churn_multinomail_bayes)
bernoulli_bayes_accuracy = accuracy_score(y_test, predicted_churn_bernoulli_bayes)

accuracy_table = pd.DataFrame({
    'Model': ['GaussianNB', 'MultinomialNB', 'BernoulliNB'],
    'Accuracy': [gaussian_bayes_accuracy, multinomail_bayes_accuracy, bernoulli_bayes_accuracy],
})

accuracy_table.head()

,Model,Accuracy
0,GaussianNB,0.755855
1,MultinomialNB,0.669979
2,BernoulliNB,0.746984


# Conclusion
As, we see, each model have similar accuracy between them. The best performant model was Gaussian Naive Bayes, while the worst performant was Multinomial Naive Bayes. There might be a slight error since we used all columns to train our data but haven't picked individual columns for each algorithm because each algorithm works best with specific data:
    
- **Bernoulli** - for binary data [columns that have "Yes"/"No" values fit best for this algorithm
- **Multinomial** - for discrete data, hence why it performs the worst out of all because we don't have discrete data here
- **Gaussian** - for data that has weight [columns like Monthly Charges and Yearly Charges fit best for this algorithm]